# 分词并去除停用词

import jieba

stopwords = set()
with open('hit_stopwords.txt', 'r', encoding='utf-8') as f:
    for line in f:
        stopwords.add(line.strip())
text = "这是一段需要进行分词的文本"
seg_list = jieba.cut(text, cut_all=False)
seg_list = list(seg_list)
filtered_words = []

for word in seg_list:
    if word not in stopwords:
        filtered_words.append(word)
print(filtered_words)

In [1]:
import jieba
import pandas as pd
# 读取停用词表
stop_words = set()
with open('hit_stopwords.txt', 'r', encoding='utf-8') as f:
    for line in f:
        stop_words.add(line.strip())
def preprocess(text):
    # 使用jieba对文本进行分词
    seg_list = jieba.cut(text, cut_all=False)

    # 去除停用词
    filtered_words = []
    for word in seg_list:
        if word not in stop_words:
            filtered_words.append(word)
    if filtered_words == []:
        filtered_words.append('无')
    return filtered_words

In [2]:
job_requirements_df = pd.read_csv('./data/result1-1.csv')
job_seekers_df = pd.read_csv('./data/result1-2.csv')


In [3]:
def academic_process(academic):
    if academic == '大专 ':
        return 2
    elif academic == '本科 ':
        return 3
    elif academic == '本科 硕士 ':
        return 4
    elif academic == '博士 ':
        return 5
    elif academic == '技工 ':
        return 1
    else:
        return 0

In [4]:
import re
def has_number(s):
    """判断字符串 s 中是否含有数字"""
    pattern = r'\d+'  # 定义匹配数字的正则表达式
    match = re.search(pattern, s)  # 在字符串中搜索匹配项
    return bool(match)  # 如果找到匹配项，返回 True，否则返回 False

In [5]:
def experience_process(experience):
    if(type(experience) == float):
        experience = str(experience)
    if experience == '无经验' :
        return 0
    elif has_number(experience):
        pattern = r"\d+"
        numbers = re.findall(pattern, experience)
        return int(numbers[0])
    else:
        return 0

In [6]:
job_seekers_academic_df = pd.DataFrame(columns=['求职者id', '学历'])
job_seekers_experience_df = pd.DataFrame(columns=['求职者id', '工作经验'])
job_seekers_academic_df['求职者id'] = job_seekers_df['求职者id']
job_seekers_experience_df['求职者id'] = job_seekers_df['求职者id']
job_seekers_academic_df['学历'] = job_seekers_df['学历'].apply(academic_process)
job_seekers_experience_df['工作经验'] = job_seekers_df['工作经验'].apply(experience_process)
# 将job_seekers_academic_df['学历']转换为int类型，并保存到job_seekers_df['学历数值']
job_seekers_df['学历数值'] = job_seekers_academic_df['学历'].astype(int)
job_seekers_df['工作经验数值'] = job_seekers_experience_df['工作经验'].astype(int)
job_seekers_df.fillna('无', inplace=True)
job_seekers_df['自我评价-分词'] = job_seekers_df['自我评价'].apply(preprocess)
job_seekers_df['自我评价-分词'] = job_seekers_df['自我评价-分词'].apply(lambda x: [i.replace('\n', '无') for i in x])
job_seekers_df.to_csv('result1-2_num.csv', index=False)

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\YHWK5\AppData\Local\Temp\jieba.cache
Loading model cost 0.361 seconds.
Prefix dict has been built successfully.


In [7]:
def re_experience_process(experience):
    if(type(experience) == float):
        experience = str(experience)
    if experience == '经验不限' or experience == '不限':
        return 0
    elif experience.isdigit():
        return int(experience)
    elif has_number(experience):
        pattern = r"\d+"
        numbers = re.findall(pattern, experience)
        return int(numbers[0])
    else:
        return 0

In [8]:
job_requirements_experience_df = pd.DataFrame(columns=['招聘信息id', '工作经验要求'])
job_requirements_experience_df['招聘信息id'] = job_requirements_df['招聘信息id']
job_requirements_experience_df['工作经验要求'] = job_requirements_df['工作经验要求'].apply(re_experience_process)
job_requirements_df['工作经验要求数值'] = job_requirements_experience_df['工作经验要求'].astype(int)
job_requirements_df['职位描述-分词'] = job_requirements_df['职位描述'].apply(preprocess)
job_requirements_df['职位描述-分词'] = job_requirements_df['职位描述-分词'].apply(lambda x: [i.replace('\n', '') for i in x])
job_requirements_df.to_csv('result1-1_num.csv', index=False)


In [ ]:

job_requirements_split_df = pd.DataFrame(columns=['职位描述-分词'])
job_seekers_split_df = pd.DataFrame(columns=['自我评价-分词'])
# read job_requirements_df '职位描述' column and preprocess it
# save the result to a new frame called '职位描述-分词'
# use apply() to apply the preprocess function to each row
job_requirements_split_df['职位描述-分词'] = job_requirements_df['职位描述'].apply(preprocess)
# use replace() to remove '/n' in the result
job_requirements_split_df['职位描述-分词'] = job_requirements_split_df['职位描述-分词'].apply(lambda x: [i.replace('\n', '') for i in x])
job_requirements_split_df.to_csv('职位描述-分词.csv', index=False)
# read job_seekers_df '自我评价' column and preprocess it
# save the result to a new frame called '自我评价-分词'
# use apply() to apply the preprocess function to each row
job_seekers_df.fillna('', inplace=True)
job_seekers_split_df['自我评价-分词'] = job_seekers_df['自我评价'].apply(preprocess)
# use replace() to remove '/n' in the result
job_seekers_split_df['自我评价-分词'] = job_seekers_split_df['自我评价-分词'].apply(lambda x: [i.replace('\n', '') for i in x])
job_seekers_split_df.to_csv('自我评价-分词.csv', index=False)




# 进行Word2vec训练，获取词向量

In [ ]:
from gensim.models import Word2Vec
# 读取csv文件
job_requirement_train = pd.read_csv('职位描述-分词.csv', header=0, delimiter=",")

# 选择特定的列，并将每行文本转化为列表
job_requirement_sentences = job_requirement_train['职位描述-分词'].apply(lambda x: x.split()).tolist()

# 设置Word2Vec模型的参数
vector_size = 100    # 设置词向量的维度
window = 5    # 设置上下文窗口的大小
min_count = 1 # 忽略出现次数低于此值的单词
workers = 4   # 设置用于训练模型的线程数

# 训练Word2Vec模型
requirement_model = Word2Vec(sentences=job_requirement_sentences, vector_size=vector_size, window=window, min_count=min_count, workers=workers)

# 获取单词的向量表示
# 获取词汇表中所有单词的列表
words = list(requirement_model.wv.index_to_key)

# 打印词汇表中的所有单词
for word in words:
    print(word)

# 保存训练好的模型
requirement_model.save('job_requirement_word2vec_model.bin')


In [ ]:
# 读取csv文件
job_seeker_train = pd.read_csv('自我评价-分词.csv', header=0, delimiter=",")

# 选择特定的列，并将每行文本转化为列表
job_seeker_sentences = job_seeker_train['自我评价-分词'].apply(lambda x: x.split()).tolist()

# 设置Word2Vec模型的参数
vector_size = 100    # 设置词向量的维度
window = 5    # 设置上下文窗口的大小
min_count = 1 # 忽略出现次数低于此值的单词
workers = 4   # 设置用于训练模型的线程数

# 训练Word2Vec模型
seeker_model = Word2Vec(sentences=job_seeker_sentences, vector_size=vector_size, window=window, min_count=min_count, workers=workers)

# 获取单词的向量表示
# 获取词汇表中所有单词的列表
words = list(seeker_model.wv.index_to_key)

# 打印词汇表中的所有单词
for word in words:
    print(word)

# 保存训练好的模型
seeker_model.save('job_seeker_word2vec_model.bin')


# 计算相似性并推荐

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# 将文本数据转化为TF-IDF特征向量
vectorizer = TfidfVectorizer()
job_desc_tfidf = vectorizer.fit_transform(job_requirements_split_df['职位描述-分词'])
self_intro_tfidf = vectorizer.transform(job_seekers_split_df['自我评价-分词'])


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# 计算自我评价和职位描述之间的相似度矩阵
similarity_matrix = cosine_similarity(self_intro_tfidf, job_desc_tfidf)

# 求出每个求职者和每个招聘公司之间的相似度得分，并将结果保存到DataFrame中
recommendations_df = pd.DataFrame(similarity_matrix, index=job_seekers_df['id'], columns=job_requirements_df['id'])

# 找到每个求职者对应的最相似的n个公司
n_recommendations = 3
top_n_companies = recommendations_df.apply(lambda row: row.nlargest(n_recommendations).index.tolist(), axis=1)

# 将推荐结果保存到CSV文件中
top_n_companies_df = pd.DataFrame({'id': job_seekers_df['id'], 'top_n_companies': top_n_companies})
top_n_companies_df.to_csv('top_n_companies.csv', index=False)
